In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Download training and test data from open datasets

In [2]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:04<00:00, 5957012.27it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 146706.47it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:07<00:00, 589170.52it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 26332045.11it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## Create data loaders

In [3]:
batch_size = 64

In [4]:
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [5]:
train_dataloader

In [6]:
for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Ceating Models

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [8]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
model.parameters

<bound method Module.parameters of NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)>

In [11]:
model.parameters()

<generator object Module.parameters at 0x7f411a7e6f20>

In [13]:
len(list(model.parameters()))

6

In [14]:
for i in model.parameters():
    print(i)
    break

Parameter containing:
tensor([[ 0.0033,  0.0273,  0.0095,  ..., -0.0124, -0.0133,  0.0237],
        [-0.0136, -0.0309, -0.0330,  ...,  0.0257,  0.0227, -0.0313],
        [-0.0228,  0.0107, -0.0223,  ..., -0.0137,  0.0195, -0.0132],
        ...,
        [-0.0287, -0.0035,  0.0032,  ..., -0.0113, -0.0201, -0.0093],
        [ 0.0113, -0.0235, -0.0014,  ..., -0.0185, -0.0119,  0.0009],
        [-0.0267, -0.0038,  0.0106,  ...,  0.0012,  0.0214, -0.0121]],
       requires_grad=True)


## Optimizing the Model Parameters

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error 
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')
            

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
--------------------------------
loss: 2.310035  [    0/60000]
loss: 2.303258  [ 6400/60000]
loss: 2.279186  [12800/60000]
loss: 2.268244  [19200/60000]
loss: 2.264480  [25600/60000]
loss: 2.223747  [32000/60000]
loss: 2.231352  [38400/60000]
loss: 2.204265  [44800/60000]
loss: 2.198878  [51200/60000]
loss: 2.165134  [57600/60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 2.161363 

Epoch 2
--------------------------------
loss: 2.173583  [    0/60000]
loss: 2.167988  [ 6400/60000]
loss: 2.114211  [12800/60000]
loss: 2.126227  [19200/60000]
loss: 2.089085  [25600/60000]
loss: 2.022301  [32000/60000]
loss: 2.047068  [38400/60000]
loss: 1.980851  [44800/60000]
loss: 1.976543  [51200/60000]
loss: 1.912954  [57600/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 1.907027 

Epoch 3
--------------------------------
loss: 1.936014  [    0/60000]
loss: 1.911894  [ 6400/60000]
loss: 1.802120  [12800/60000]
loss: 1.838289  [19200/60000]
loss: 1.734155  [25600/60000]
loss: 1.682503  [32000/

## Saving Models

In [24]:
torch.save(model.state_dict(), 'mymodel.pth')
print('Saved Pytorch Model State to mymodel.pth')

Saved Pytorch Model State to mymodel.pth


## Loading Models

In [25]:
model = NeuralNetwork()
model.load_state_dict(torch.load('mymodel.pth'))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[10][0], test_data[10][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Coat"
